In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movie = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movie.head()

In [ ]:
movie = movie.merge(credits, on='title')

In [ ]:
movie = movie[['id', 'title', 'genres', 'keywords', 'overview', 'cast', 'crew']]

In [ ]:
movie.isnull().sum()

In [ ]:
movie.dropna(inplace=True)

In [ ]:
movie.duplicated().sum()

In [ ]:
movie['genres'][0]

In [ ]:
import ast

In [ ]:
def func1(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    
    return L

In [ ]:
movie['genres'] = movie['genres'].apply(func1)

In [ ]:
movie.head()

In [ ]:
movie['keywords'][0]

In [ ]:
def func2(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    
    return L


In [ ]:
movie['keywords'] = movie['keywords'].apply(func2)

In [ ]:
movie['cast'][0]

In [ ]:
def func3(obj):
    L = []
    count = 0
    for i in ast.literal_eval(obj):
        if count != 3:
            L.append(i['name'])
            count += 1
        else:
            break
    
    return L

In [ ]:
movie['cast'] = movie['cast'].apply(func3)

In [ ]:
movie['crew'][0]

In [ ]:
def func4(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    
    return L

In [ ]:
movie['crew'] = movie['crew'].apply(func4)

In [ ]:
movie['genres'] = movie['genres'].apply(lambda x:[i.replace(" ", "") for i in x])
movie['keywords'] = movie['keywords'].apply(lambda x:[i.replace(" ", "") for i in x])
movie['cast'] = movie['cast'].apply(lambda x:[i.replace(" ", "") for i in x])
movie['crew'] = movie['crew'].apply(lambda x:[i.replace(" ", "") for i in x])

In [ ]:
movie['overview'] = movie['overview'].apply(lambda x:x.split())

In [ ]:
movie['tags'] = movie['overview'] + movie['genres'] + movie['keywords'] + movie['cast'] + movie['crew']

In [ ]:
new_df = movie[['id', 'title', 'tags']]

In [ ]:
new_df.head()

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df['tags'][0]

In [ ]:
pip install nltk

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(val):
    new = []
    for i in val.split():
        new.append(ps.stem(i))
        
    return " ".join(new)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
new_df['tags'][0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [ ]:
def recommend(mov):
    movie_index = new_df[new_df['title'] == mov].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]
    for movie in movie_list:
        print(new_df.iloc[movie[0]].title)

In [ ]:
recommend('Batman Begins')

In [72]:
import pickle

In [75]:
pickle.dump(new_df.to_dict(), open('Movie_list.pkl', 'wb'))

In [77]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))